# Comet.ml visualisation

[comet.ml](https://www.comet.ml) is a nice experiment tracking service, please sign-up before proceeding to the exercise.
Also you would need API key you can get from 'Settings' -> 'Account settings' that we save to a config file.


In [0]:
%%writefile .comet.config
[comet]
api_key=<YOUR KEY>
logging_file = /tmp/comet.log
logging_file_level = info


In [0]:
! [ ! -z "$COLAB_GPU" ] && pip install skorch comet_ml

In [0]:
from comet_ml import Experiment
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(0);


## Training a classifier and making predictions

### A toy binary classification task

We load a toy classification task from `sklearn`.

In [0]:
import numpy as np
from sklearn.datasets import make_classification

In [0]:
X, y = make_classification(1000, 20, n_informative=10, n_classes=2, random_state=0)
X = X.astype(np.float32)

In [0]:
X.shape, y.shape, y.mean()

### Defining and training the neural net classifier

In [0]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

In [0]:
from skorch import NeuralNetClassifier

In [0]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    module__num_units=10,
    lr=0.1,
    device='cuda',  # comment this to train with CPU
)

Create Experiment instance for various training info

In [0]:
experiment = Experiment(project_name="comet-4")

In [0]:
experiment.log_parameters({'num_units': 10})

In [0]:
net.fit(X, y)

In [0]:
experiment.log_metrics({'valid_loss': net.history[-1, 'valid_loss']})
experiment.end()

Now go check comet page / plots

## Task
try a few more experiments with `num_units` in [20, 30, 40] 

In [0]:
for num_units in <YOUR CODE>: # range from 20 to 30 with step 10
  net = NeuralNetClassifier(
      ClassifierModule,
      max_epochs=20,
      module__num_units=num_units,
      lr=0.1,
      device='cuda',  # comment this to train with CPU
  )
  experiment = Experiment(project_name="comet-4")
  experiment.log_parameters({'num_units': num_units})
  net.fit(X, y)
  experiment.log_metrics({'valid_loss': net.history[-1, 'valid_loss']})
  experiment.end()

Now go and check comet.ml page / plots